In [1]:
import numpy as np
import pandas as pd
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import drGAT

# model evaluation

In [3]:
test = torch.load('test.pt')

In [4]:
tmp = !ls | grep pt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('sample.pt', map_location=device)

In [6]:
drGAT.eval(model, test)

,Accuracy,Precision,Recall,F1 Score,True Positive,True Negative,False Positive,False Negative
0,0.771375,0.740881,0.783245,0.761474,1178,1312,412,326
